In [ ]:
import pandas as pd
import numpy as np

In [ ]:
posts = pd.read_csv("posts.csv", sep=r'\s*,\s*', header=0, engine='python')
views = pd.read_csv("views.csv")
users = pd.read_csv("users.csv")

In [ ]:
posts.head()

In [ ]:
views.head()

In [ ]:
users.head()
print(posts.columns.tolist())

In [ ]:
posts = posts.rename(columns = {"_id" : "post_id"})
users = users.rename(columns = { "_id": "user_id"})

In [ ]:
posts.head()

In [ ]:
users.head()

In [ ]:
merged_df = users.merge(views, on = "user_id")

In [ ]:
merged_df = merged_df.merge(posts, on = "post_id")

In [ ]:
merged_df.head()

In [ ]:
merged_df.drop(["timestamp"], axis = 1, inplace = True)

In [ ]:
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
cleaned_df = merged_df.dropna()

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df["post_type"].describe()

In [ ]:
def combined_features(row):
    return row["title"] +" "+row["category"] +" "+row["post_type"] 

cleaned_df["combined_features"] = cleaned_df.apply(combined_features, axis = 1)

In [ ]:
cleaned_df["combined_features"].head()

In [ ]:
cleaned_df_ = cleaned_df.drop_duplicates(subset=["name"])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

In [ ]:
tfv_matrix = tfv.fit_transform(cleaned_df_["combined_features"])

In [ ]:
tfv_matrix

In [ ]:
tfv_matrix.shape

In [ ]:
cleaned_df["name"].describe()

In [ ]:
cleaned_df_ = cleaned_df_.reset_index()

In [ ]:
cleaned_df_.head()

In [ ]:
cleaned_df_.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig= sigmoid_kernel(tfv_matrix, tfv_matrix)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfv_matrix, tfv_matrix)

In [ ]:
indices = pd.Series(cleaned_df_.index, index=cleaned_df_['name'])

In [ ]:
indices

In [ ]:
indices.shape

In [ ]:
indices["Shivani Gaba"]

In [ ]:
sig[indices["Shivani Gaba"]]

In [ ]:
cos_sim[indices["Shivani Gaba"]]

In [ ]:
list(enumerate(sig[indices["Shivani Gaba"]]))

In [ ]:
list(enumerate(cos_sim[indices["Shivani Gaba"]]))

In [ ]:
sorted_cos_sim = sorted(list(enumerate(cos_sim[indices["Shivani Gaba"]])), key=lambda x: x[1], reverse=True)

In [ ]:
def give_rec(name, sig=sig):
    # Get the index corresponding to original_title
    idx = int(indices[name])

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    index = [i[0] for i in sig_scores]
    
    result = (cleaned_df_['title'].iloc[index]).drop_duplicates()
    # Scores of the top k non_duplicate most similar posts
    result = result[1:6]

    return result

In [ ]:
print(give_rec("Nachiket Khatri"))

In [ ]:
cleaned_df_.shape